### ISLP Chapter 13 - Multiple Testing

#### Applied exercise 7

In [1]:
# import libraries
import numpy as np
import pandas as pd
import statsmodels.api as sm
import scipy.stats as stats

In [2]:
# import data visualisation tools
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
plt.style.use('seaborn-v0_8-whitegrid')

import warnings
warnings.filterwarnings('ignore')

plt.rcParams['figure.figsize'] = (10, 8)

In [3]:
# read Carseats dataset
file = "../Data/Carseats.csv"
carseats = pd.read_csv(file, index_col=[0])

In [4]:
carseats.head()

,Sales,CompPrice,Income,Advertising,Population,Price,ShelveLoc,Age,Education,Urban,US
SlNo,,,,,,,,,,,
1,9.50,138,73,11,276,120,Bad,42,17,Yes,Yes
2,11.22,111,48,16,260,83,Good,65,10,Yes,Yes
3,10.06,113,35,10,269,80,Medium,59,12,Yes,Yes
4,7.40,117,100,4,466,97,Medium,55,14,Yes,Yes
5,4.15,141,64,3,340,128,Bad,38,13,Yes,No


In [5]:
carseats.info()

<class 'pandas.core.frame.DataFrame'>
Index: 400 entries, 1 to 400
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Sales        400 non-null    float64
 1   CompPrice    400 non-null    int64  
 2   Income       400 non-null    int64  
 3   Advertising  400 non-null    int64  
 4   Population   400 non-null    int64  
 5   Price        400 non-null    int64  
 6   ShelveLoc    400 non-null    object 
 7   Age          400 non-null    int64  
 8   Education    400 non-null    int64  
 9   Urban        400 non-null    object 
 10  US           400 non-null    object 
dtypes: float64(1), int64(7), object(3)
memory usage: 37.5+ KB


In [6]:
# select numeric variables from dataset
cols_num = carseats.select_dtypes(include='number').columns.tolist()
cols_num

['Sales',
 'CompPrice',
 'Income',
 'Advertising',
 'Population',
 'Price',
 'Age',
 'Education']

#### 7.a

#### Fit a linear model with `Sales` as response variable and each of the numeric variables. Report the `p-values` associated with the coefficients for the variables.

In [7]:
# drop 'Sales' from cols_num
# cols_num.pop(0)
# cols_num

In [8]:
# perform linear regression models
# y = carseats['Sales']

# for col in cols_num:
#     print(col)
#     X = carseats[col]
#     X = sm.add_constant(X)
#     reg = sm.OLS(y, X).fit()
#     print(reg.summary())
#     print('\n')

In [9]:
# subset dataset with numeric variables only
carseats_num = carseats[cols_num].set_index('Sales')
carseats_num

,CompPrice,Income,Advertising,Population,Price,Age,Education
Sales,,,,,,,
9.50,138,73,11,276,120,42,17
11.22,111,48,16,260,83,65,10
10.06,113,35,10,269,80,59,12
7.40,117,100,4,466,97,55,14
4.15,141,64,3,340,128,38,13
...,...,...,...,...,...,...,...
12.57,138,108,17,203,128,33,14
6.14,139,23,3,37,120,55,11
7.41,162,26,12,368,159,40,18


In [10]:
# Source: https://stackoverflow.com/questions/55719276/linear-regression-on-each-column-without-creating-for-loops-or-functions
lreg_df = carseats_num.apply(lambda x: stats.linregress(carseats_num.index, x), result_type='expand') \
    .rename(index={
        0: 'slope', 
        1: 'intercept', 
        2: 'rvalue', 
        3: 'p-value', 
        4: 'stderr'}
        ) \
    .T \
    .sort_values(by='p-value') \
    .round(4)

lreg_df

,slope,intercept,rvalue,p-value,stderr
Price,-3.7304,143.7589,-0.4450,0.0000,0.3763
Advertising,0.6346,1.8775,0.2695,0.0000,0.1137
Age,-1.3298,63.2910,-0.2318,0.0000,0.2797
Income,1.5058,57.3697,0.1520,0.0023,0.4910
CompPrice,0.3479,122.3667,0.0641,0.2009,0.2716
Education,-0.0482,14.2614,-0.0520,0.2999,0.0464
Population,2.6338,245.0960,0.0505,0.3140,2.6125


#### 7.b

Using an $alpha$ = **0.05**, `CompPrice`, `Population` and `Education` `p-values` are greater than alpha (not statistically significant). The other variables `p-values` are less than alpha (statistically significant). In other words, we reject the null hypothesis (i.e., coefficients are not statistically significant) for variables `Price`, `Advertising`, `Age` and `Income`.

#### 7.c

#### Now suppose we control the FWER at level 0.05 for the p-values. Which null hypotheses do we reject?

In [11]:
from statsmodels.stats.multitest import multipletests as mult_test

In [14]:
lreg_df['p-value']

Price          0.0000
Advertising    0.0000
Age            0.0000
Income         0.0023
CompPrice      0.2009
Education      0.2999
Population     0.3140
Name: p-value, dtype: float64

In [22]:
# FWER with Bonferroni correction
reject, bonf = mult_test(lreg_df['p-value'].to_list(), alpha=0.05, method='bonferroni')[:2]
print(reject)
print(bonf)

[ True  True  True  True False False False]
[0.     0.     0.     0.0161 1.     1.     1.    ]


In [24]:
# FWER with Holm correction
reject, bonf = mult_test(lreg_df['p-value'].to_list(), alpha=0.05, method='holm')[:2]
print(reject)
print(bonf)

[ True  True  True  True False False False]
[0.     0.     0.     0.0092 0.6027 0.6027 0.6027]


#### 7.d

#### Finally, suppose we control the FDR at level 0.2 for the p-values. Which null hypotheses do we reject?

In [33]:
# FDR (Benjamini--Hochberg procedure)
q_values = mult_test(lreg_df['p-value'].to_list(), method='fdr_bh')[1]
q_values

array([0.      , 0.      , 0.      , 0.004025, 0.28126 , 0.314   ,
       0.314   ])

In [34]:
(q_values <= 0.2).sum()

4